In [1]:
!nvidia-smi

Sun Jul 26 13:29:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import auth
auth.authenticate_user()

In [3]:
!gcloud config set project lively-tensor-284509

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



In [4]:
!gsutil -m cp -r gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/ /content/

Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/._SUCCESS.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-00000-a90c94c1-0260-4d76-acad-e693121544fe-c000.snappy.parquet.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-00001-a90c94c1-0260-4d76-acad-e693121544fe-c000.snappy.parquet.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-00002-a90c94c1-0260-4d76-acad-e693121544fe-c000.snappy.parquet.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-00003-a90c94c1-0260-4d76-acad-e693121544fe-c000.snappy.parquet.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-00004-a90c94c1-0260-4d76-acad-e693121544fe-c000.snappy.parquet.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-00005-a90c94c1-0260-4d76-acad-e693121544fe-c000.snappy.parquet.crc...
Copying gs://nlp_recsys2020/recsys2020_data/user_sampled.parquet/.part-0

In [5]:
!pip install transformers torch dask dask[array] dask[dataframe] numpy pandas pyarrow tqdm

     |████████████████████████████████| 778kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 13.2MB/s 
     |████████████████████████████████| 890kB 54.7MB/s 
     |████████████████████████████████| 1.1MB 43.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=2ccb5c1412ecaa5c55ce775d46dca0f111d7963df72458112e858c416f7dbbf0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [10]:
import numpy as np
import pyarrow
import pyarrow.parquet as pq
from pathlib import PosixPath
from functools import partial
from transformers import DistilBertModel, DistilBertConfig, DistilBertTokenizer, BertTokenizer, BertModel
import torch
import tqdm
import shutil
import pandas as pd
import dask
import dask.array as da
import dask.dataframe as dd
import math
import torch.multiprocessing as mp

max_token_len = 600
arch = 'distilbert'
# arch = 'bert'

def embed_partition(df, d, model, batch_size) -> dd.DataFrame:
    num_rows = df.shape[0]
    num_batches = int(math.ceil(num_rows / batch_size))
    res = []
    with tqdm.tqdm(total=num_rows, position=0) as p:
        for b_idx in range(num_batches):
            int_col = df.loc[b_idx*batch_size:min(num_rows-1, (b_idx+1)*batch_size-1), 'tokens']
            # # GPU-side padding is even slower:
            # tensors = [torch.tensor(i).to(d) for i in int_col]
            # input_ids = torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)
            # input_ids = input_ids.long()

            # CPU/Numpy seems to be better at padding:
            pad_len = min(max_token_len, max(len(a) for a in int_col))
            int_col = np.vstack([np.pad(i, (0, pad_len - len(i),), constant_values=(1,)) if len(i) <
                    pad_len else i[:pad_len] for i in int_col])
            input_ids = torch.tensor(int_col).to(d).long()
            mask = (input_ids != 0).unsqueeze(2)
            outputs =   (
                    (torch.stack(model(input_ids)[2 if arch == 'bert' else 1][:-2], dim=0).sum(axis=0) * mask)
                    .sum(axis=1) / mask.sum(axis=1)
                    ).float().cpu()
            res.extend(outputs[i].numpy() for i in range(outputs.shape[0]))
            p.update(len(int_col))
    df = pd.concat([df, pd.DataFrame(res, columns=['e{:03d}'.format(i) for i in range(768)], index=df.index, dtype=np.float32)], axis='columns')
    return df

def compute_embeddings(batch_size, data_dir, ds_name):
    with dask.config.set(scheduler='synchronous'):
        data_dir = PosixPath(data_dir)
        ds_name = "user_sampled"
        input_file = data_dir / f"{ds_name}.parquet/"
        output_file = data_dir / f"{ds_name}_embeddings.parquet/"
        df = dd.read_parquet(str(input_file))

        meta = pd.concat([df._meta, pd.DataFrame(columns=['e{:03d}'.format(i) for i in range(768)], dtype=np.float32)])
        d = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        with torch.no_grad():
            if arch == 'distilbert':
                model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased', output_hidden_states=True)
            elif arch == 'bert':
                model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states=True)
            model = model.eval().to(d)
            df = df.map_partitions(embed_partition, d=d, model=model, meta=meta, batch_size=batch_size)
            df.to_parquet(output_file)


In [11]:
compute_embeddings(batch_size=200, data_dir="/content/", ds_name='user_sampled')

/usr/local/lib/python3.6/dist-packages/torch/storage.py:34: FutureWarning: pickle support for Storage will be removed in 1.5. Use `torch.save` instead
  warnings.warn("pickle support for Storage will be removed in 1.5. Use `torch.save` instead", FutureWarning)
100%|██████████| 5343/5343 [00:04<00:00, 1156.74it/s]


In [12]:
!ls -lah /content/user_sampled_embeddings.parquet

total 6.0G
drwxr-xr-x 2 root root  12K Jul 26 14:15 .
drwxr-xr-x 1 root root 4.0K Jul 26 13:34 ..
-rw-r--r-- 1 root root  97K Jul 26 14:15 _common_metadata
-rw-r--r-- 1 root root  18M Jul 26 14:15 _metadata
-rw-r--r-- 1 root root  24M Jul 26 13:47 part.0.parquet
-rw-r--r-- 1 root root  24M Jul 26 14:00 part.100.parquet
-rw-r--r-- 1 root root  24M Jul 26 13:59 part.101.parquet
-rw-r--r-- 1 root root  24M Jul 26 14:12 part.102.parquet
-rw-r--r-- 1 root root  24M Jul 26 13:48 part.103.parquet
-rw-r--r-- 1 root root  24M Jul 26 14:09 part.104.parquet
-rw-r--r-- 1 root root  23M Jul 26 14:06 part.105.parquet
-rw-r--r-- 1 root root  24M Jul 26 14:11 part.106.parquet
-rw-r--r-- 1 root root  24M Jul 26 14:13 part.107.parquet
-rw-r--r-- 1 root root  25M Jul 26 14:07 part.108.parquet
-rw-r--r-- 1 root root  23M Jul 26 13:55 part.109.parquet
-rw-r--r-- 1 root root  24M Jul 26 13:53 part.10.parquet
-rw-r--r-- 1 root root  24M Jul 26 13:57 part.110.parquet
-rw-r--r-- 1 root root  24M Jul 26 14:14 p

In [13]:
!gsutil -m cp -r /content/user_sampled_embeddings.parquet gs://nlp_recsys2020/recsys2020_data/user_sampled_embeddings.parquet/

Copying file:///content/user_sampled_embeddings.parquet/part.254.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.179.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.112.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.183.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.15.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.82.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.139.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.108.parquet [Content-Type=application/octet-stream]...
Copying file:///content/user_sampled_embeddings.parquet/part.56.parquet [C